In [ ]:
# 将 llm_toy/src 加入 sys.path（稳健多候选）
import sys as _sys
from pathlib import Path as _Path
def _add_src_path():
    cands = [
        _Path.cwd()/'llm_toy'/'src',
        _Path.cwd()/'src',
        _Path.cwd().parent/'llm_toy'/'src',
        _Path.cwd().parent/'src',
    ]
    for base in list(_Path.cwd().parents)[:3]:
        cands += [base/'llm_toy'/'src', base/'src']
    for p in cands:
        if (p/'model.py').exists() and (p/'utils.py').exists():
            _sys.path.append(str(p.resolve()))
            print('已添加src路径:', p.resolve())
            return
    print('警告：未找到 llm_toy/src，请手动添加路径或调整工作目录。')
_add_src_path()


# 06 Tokenization基础：从零训练一个BPE Tokenizer

使用 tokenizers 在toy语料上训练简易BPE Tokenizer，并与GPT-2 Tokenizer对比。

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.processors import TemplateProcessing
from transformers import GPT2Tokenizer
from pathlib import Path as _P2
_cwd = _P2.cwd()
c1 = _cwd/'llm_toy'/'data'/'tiny_corpus.txt'
c2 = _cwd/'data'/'tiny_corpus.txt'
c3 = _cwd.parent/'llm_toy'/'data'/'tiny_corpus.txt'
data_path = c1 if c1.exists() else (c2 if c2.exists() else c3)
save_path = (_cwd/'llm_toy'/'data'/'bpe_tokenizer.json')
(data_path.exists(), str(data_path))


In [ ]:
tokenizer = Tokenizer(BPE(unk_token='[UNK]'))
tokenizer.pre_tokenizer = ByteLevel()
trainer = BpeTrainer(vocab_size=2000, special_tokens=['[PAD]','[UNK]','[BOS]','[EOS]'])
tokenizer.train(files=[str(data_path)], trainer=trainer)
tokenizer.post_processor = TemplateProcessing(
    single='[BOS] $A [EOS]',
    pair='[BOS] $A [EOS] $B:1 [EOS]:1',
    special_tokens=[('[BOS]', tokenizer.token_to_id('[BOS]')), ('[EOS]', tokenizer.token_to_id('[EOS]'))]
)
tokenizer.save(str(save_path))
(str(save_path), save_path.exists())


In [ ]:
sample = '自然语言处理让计算机理解文本，Tokenization是第一步。'
enc = tokenizer.encode(sample)
(enc.tokens, enc.ids, tokenizer.decode(enc.ids))


In [ ]:
gpt2_tok = GPT2Tokenizer.from_pretrained('gpt2')
g_ids = gpt2_tok.encode(sample)
g_tokens = gpt2_tok.convert_ids_to_tokens(g_ids)
g_tokens[:20]
